# Big Q

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

np.random.seed(42)

In [11]:
#X, y = load_iris(return_X_y=True)

# Load data
columns = ['q'+ str(i) for i in range(10)] + ['Age', 'Gender', 'Ethnicity', 'Born', 'Family', 'Who', 'Country', 'Used', 'Group','Screening', 'Score']
data = pd.read_csv('autism_data.arff', sep=',', header=None)
data.columns = columns
data.head()

# Create X and Y data
X = data.iloc[:,:6]
y = data.iloc[:,-1]
y = np.double(y=="YES")
print(X.shape, y.shape)

# pick inputs and labels from the first two classes only,
# corresponding to the first 100 samples
X = X[:100]
y = y[:100]

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

(704, 6) (704,)


In [12]:
y_scaled

array([-1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,  1.,
        1.,  1., -1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,  1.,
       -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1.,
       -1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,
       -1., -1.,  1.,  1., -1., -1., -1., -1., -1.])

In [13]:
n_qubits = len(X_train[0])
n_qubits

6

In [14]:
dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [15]:
kernel(X_train[0], X_train[0])

tensor(1., requires_grad=True)

In [16]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [17]:
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [18]:
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

0.88

In [19]:
dev_kernel.num_executions

7501